<center><h1> Usando Modelos de Deep Learning com apoio do OpenCV </h1></center>
<center><h1> Classificação de Imagens </h1></center>

<h2>1 - As bibliotecas/pacotes pré-requisitos para este workshop são:</h2>

<ul>
    <li>OpenCV</li>
    <li>Matplotlib</li>
    <li>Numpy</li>
</ul>
<h3>Inicialmente, será necessário atualizar a versão do OpenCV para 4.5 ou maior.
<ul>
    <li>Executar a célula abaixo (que contém o comando 'pip install opencv-python --upgrade')</li>
 
</ul>



In [ ]:
!pip install opencv-python --upgrade

<ul>
    <li>Importar a biblioteca do OpenCV e confirmar a versão que está sendo usada é 4.5 ou maior.</li>
 
</ul>


In [ ]:
# Importar o OpenCV
import cv2
print("OpenCV version:", cv2.__version__)

In [ ]:
# Importar as demais bibliotecas/pacotes necessários ao projeto

import matplotlib.pyplot as plt
import numpy as np
import os
import requests
from google.colab.patches import cv2_imshow 

## Acesso ao google drive a partir do colab
from google.colab import drive
#drive.mount('/content/gdrive') 
drive.mount("/content/gdrive", force_remount=True)

## IMPORTANTE:
## Pré-requisitos para a configuração de acesso aos recursos do workshop:
## 1. A pasta 'recursos_workshop' deve ter sido compartilhada com o seu usuário do Google Drive
## 2. O aluno deverá fazer acesso à pasta compartilhada em sua conta de Google Drive
## 3. O aluno deverá criar um atalho (shortcut) para a pasta compartilhada. 
##    Este atalho ficará localizado no próprio drive do aluno ('Meu Drive' ou 'MyDrive') e
##    terá o nome 'recursos_workshop'


## Caminho para a pasta de recursos do workshop
resources_path = "gdrive/MyDrive/recursos_workshop/"

## Caminho para a pasta de modelos de Deep Learning
models_path = resources_path + "modelos_DL/"

## Caminho para a pasta de imagens 
image_dir = resources_path + "imagens"



## 2 - Configurações: diretórios de modelos, imagens, notebooks 
#### definições/configurações de modelos de Deep Learning e pesos pré-treinados: 
<ul>
<li>Estão localizados na pasta 'recursos_workshop' do Google drive</li>
</ul>

#### imagens:
<ul>  
    <li>Também estão no google drive ou, dependendo do caso, serão baixadas da internet</li>
</ul>
    


## 3 - Leitura de imagens e pré-processamento 
### 3.1 - A imagem pode ser lida do disco usando o método "imread" do OpenCV
<ul>
    <li>cv2.imread</li>    
</ul>

### 3.2 - O método "blobFromImage" permite fazer resize, crop, scaling, normalizing, mudar de RGB para BGR.
### 3.3 - Este método produz um BLOB de 4 dimensões 
<ul>
    <li>cv2.dnn.blobFromImage</li>
    <li>cv2.dnn.blobFromImages</li>    
</ul>


## 4 - Frameworks e modelos
### 4.1 - Os seguintes frameworks são suportados pelo módulo <a href="https://github.com/opencv/opencv/tree/master/modules/dnn" target="_blank" rel="noopener noreferrer">DNN</a> do OpenCV

<ul>
<li><a href="http://caffe.berkeleyvision.org/" target="_blank" rel="noopener noreferrer">Caffe</a></li>
<li><a href="https://www.tensorflow.org/" target="_blank" rel="noopener noreferrer">Tensorflow</a></li>
<li><a href="http://torch.ch/" target="_blank" rel="noopener noreferrer">Torch</a></li>
<li><a href="https://pjreddie.com/darknet/" target="_blank" rel="noopener noreferrer">Darknet</a></li>
<li><a href="https://onnx.ai/" target="_blank" rel="noopener noreferrer">ONNX</a></li>
</ul>    


## 5 - Carga em memória de modelos de DNN (a partir de modelo serializado em disco)
### OpenCV usa modelos pré-treinados em datasets com acesso público (por exemplo, ImageNet). Esses modelos são desenvolvidos com o uso de diversos frameworks (Caffe, Tensorflow, Pytorch, etc.)
<ul>
    <li>cv2.dnn.readNetFromCaffe</li>
    <li>cv2.dnn.readNetFromDarknet</li>
    <li>cv2.dnn.readNetFromTensorFlow</li>
    <li>cv2.dnn.readNetFromTorch</li>
    <li>cv2.dnn.readNetFromONNX</li>
</ul>

## 6 - O processo de inferência usando o modelo
### 6.1 - Definindo o input para o modelo e iniciando a inferência

<ul>
<li>setInput(blob)</li>
<li>forward</li>
</ul>

## 7 - Classificação de imagens
### A classificação é a tarefa de categorizar uma imagem com base em um conjunto pré-definido de classes.
### Logo abaixo, iremos definir algumas funções de apoio para as nossas tarefas.

### 7.1 - Carregar o modelo <a href="https://arxiv.org/abs/1409.4842" target="_blank" rel="noopener noreferrer">GoogLeNet</a> (desenvolvido com uso do framework Caffe) na memória

In [ ]:
def loadNetModel(caffe_dir, caffe_model, caffe_proto):
    model = os.path.join(caffe_dir, caffe_model)
    proto = os.path.join(caffe_dir, caffe_proto)
    net = cv2.dnn.readNetFromCaffe(proto, model)
    return net

### 7.2 - Obter os nomes das classes associadas às predições a partir do dataset 'classes_file'

In [ ]:
def getClassNames(caffe_dir, classes_file):
    labs_fpath = os.path.join(caffe_dir, classes_file)
    cfile = open(labs_fpath)
    file_rows = cfile.read().strip().split("\n")
    classes = [row[row.find(" ") + 1:].split(",")[0] for row in file_rows]
    return classes

### 7.3 - Utilizando OpenCV, ler a imagem do arquivo 'image_file'  do diretório 'image_dir'

In [ ]:
def getImageAndBlob(image_dir, image_file):
    # Read image from disk
    image = cv2.imread(os.path.join(image_dir, image_file))
    # Pre-process image and obtain blob
    blob = cv2.dnn.blobFromImage(image, 1, (224, 224), (104, 117, 123))
    return image, blob 

### 7.4 - Definir a imagem lida como entrada para o processamento do modelo

In [ ]:
def setNetInput(net, blob):
    #set input 
    net.setInput(blob)
    return net

### 7.5 - Processar a inferência neste modelo, usando esta imagem 

In [ ]:
# O blob da imagem a ser usado como input para o modelo realizar a inferência
# precisa já ter sido definido antes de acionar esta função
def processInference(net):
    #forward
    predictions = net.forward()
    return predictions

### 7.6 - As predições do modelo apresentam um índice de confiança como resultado. Ordenar em ordem descendente essas predições e obter as 'qtd_preds' com maior índice de confiança

In [ ]:
def getOrderedPreds(predictions, qtd_preds):
    # argsort ordena em ordem ascendente
    ind_sorted = np.argsort(predictions[0])
    # obter o tamanho do slice que contém as últimas 'qtd_preds' desejadas
    last = len(predictions[0])
    lastn = last - qtd_preds
    # obter as últimas 'qtd_preds' da lista em ordem descendente
    ordered_indexes = ind_sorted[lastn: last][::-1]
    return ordered_indexes

### 7.7 - Mostrar as predições com maior índice de confiança

In [ ]:
# exibe as predições com maior índice de confiança
def printPredictions(ordered_indexes, classes, predictions, image_file):
    print("Índice de confiança das predições de classes para a imagem:", image_file)
    print("Ranking\t\tclasse\t\t\tconf.index\t\t\tclassnum")
    for (i, idx) in enumerate(ordered_indexes):
        print("{:<10} \t{:<20} \t{:.5} \t\t\t{:<10}".format(i + 1, classes[idx], predictions[0][idx], str(idx)))


### 7.8 - Inserir na imagem o nome da classe que apresentou o maior índice de confiança

In [ ]:
def putTextInImage(image, ordered_indexes, classes, predictions):
    itop = ordered_indexes[0]
    text = "Classe: {}, Conf: {:.2f}%".format(classes[itop], predictions[0][itop] * 100)
    cv2.putText(image, text, (3, 15),  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (10, 10, 255), 2) 
    return image            

### 7.9 - Exibir a imagem usando OpenCV

In [ ]:
def imageResize(frame, maxH):
    h, w = frame.shape[:2] # obter a altura e largura do frame
    if maxH < h:
        aspect_ratio = w/h
        blob_height = maxH 
        blob_width = int(blob_height * aspect_ratio)
        dsize = (blob_width, blob_height) # keep the frame's original aspect ratio
        return cv2.resize(frame, dsize)
    return frame

def showImage(image):
    heightMax = 600  # max height used here to do a imshow inside Colab
    out = imageResize(image, heightMax)
    cv2_imshow(out)


### 7.10 - Implementação da classificação da imagem contida em 'image_file' e exibição dos resultados

In [ ]:
def classifyAndDisplayImage(net, classes, image_dir, image_file, qtd_preds):

    # Utilizando o framework Caffe, o modelo GoogLeNet e os nomes das classes associadas às predições,
    # processar a classificação da imagem informada

    # 7.3 - Utilizando OpenCV, ler a imagem do arquivo 'image_file' do diretório 'image_dir'
    image, blob = getImageAndBlob(image_dir, image_file)


    # 7.4 - Definir a imagem lida como entrada para o processamento do modelo
    net = setNetInput(net, blob)


    # 7.5 - Processar a inferência neste modelo, usando esta imagem
    predictions = processInference(net)


    # 7.6 - As predições do modelo apresentam um índice de confiança como resultado. Ordenar em ordem descendente essas predições e obter as 'qtd_preds' com maior índice de confiança
    ordered_indexes = getOrderedPreds(predictions, qtd_preds)

    # 7.7 - Mostrar as predições com maior índice de confiança
    printPredictions(ordered_indexes, classes, predictions, image_file)


    # 7.8 - Inserir na imagem o nome da classe que apresentou o maior índice de confiança
    image = putTextInImage(image, ordered_indexes, classes, predictions)

    # 7.9 - Exibir a imagem usando OpenCV
    showImage(image)

    

### 7.11 - Acionamento do processo de classificação de imagens utilizando as funções de apoio
#### 7.11.1 - Definições de configuração: diretórios de modelos, imagens; arquivos de modelos e classes


In [ ]:
# Definições de diretórios, arquivos de modelo, pesos

###  A definição do diretório de imagens está no início deste notebook


# definir o diretório onde está armazenado o modelo a ser usado
caffe_dir = models_path + 'Googlenet'

# definir o arquivo que contém os pesos pré-treinados deste modelo 
caffe_model = 'bvlc_googlenet.caffemodel'

# definir o arquivo que contém a 'configuração' deste modelo 
caffe_proto = 'bvlc_googlenet.prototxt'

# definir o arquivo que contém as classes utilizadas para treinar o modelo
classes_file = "synset_words.txt"
#
#

#### 7.11.2 - Fazer a carga do modelo na memória e obter as classes (rótulos) utilizadas no treinamento do modelo


In [ ]:
# Inicialmente, fazer a carga do modelo na memória e obter as classes (rótulos) utilizadas
# no treinamento do modelo

# 7.1 - Utilizando o framework Caffe, carregar o modelo GoogLeNet na memória
net = loadNetModel(caffe_dir, caffe_model, caffe_proto)

# 7.2 - Obter os nomes das classes associadas às predições a partir do dataset 'classes_file'
classes = getClassNames(caffe_dir, classes_file)

# definir quantidade de predições que queremos mostrar, neste caso queremos exibir as 3 predições com maior índice de confiança
qtd_preds = 3
#

#### 7.11.3 - Definir a imagem a ser processada e acionar o processo de classificação

In [ ]:
# definir a imagem a ser processada
image_file = 'eagle.png' 

# Processar a classificação da imagem contida no arquivo 'image_file' 
classifyAndDisplayImage(net, classes, image_dir, image_file, qtd_preds)


### Atividade para o aluno
#### 7.11.4 - Exercício: utilizando o modelo já carregado, efetuar a classificação para as seguintes imagens
<ul>
<li>pexels-david-dibert-635499.jpg</li>
<li>pexels-engin-akyurt-1769271.jpg</li>
<li>pexels-frank-grün-3757197.jpg</li>
<li>pexels-pascal-renet-1089304.jpg</li>
<li>pexels-pixabay-53114.jpg</li>
</ul>

### 7.12 - Outro exemplo: classificação de imagens usando o framework Darknet
#### Primeiramente, vamos definir algumas funções de apoio

In [ ]:
def darknetGetClassNames(darknet_dir, classes_file):
    labs_fpath = os.path.join(darknet_dir, classes_file)
    cfile = open(labs_fpath)
    file_rows = cfile.read().strip().split("\n")
    classes = [row for row in file_rows] #[row[row.find(" ") + 1:].split(",")[0] for row in file_rows]
    return classes

In [ ]:
def darknetGetImageAndBlob(image_dir, image_file):
    imagec = cv2.imread(os.path.join(image_dir, image_file))
    # Pre-process image and obtain blob
    blobc = cv2.dnn.blobFromImage(imagec,  1/255.0, (256, 256), swapRB=True, crop=False)
    return imagec, blobc


In [ ]:
def loadDarknetClassifModel(darknet_dir, darknet_model, darknet_proto):
    model = os.path.join(darknet_dir, darknet_model)
    proto = os.path.join(darknet_dir, darknet_proto)
    dnet = cv2.dnn.readNetFromDarknet(proto, model)
    dnet.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    return dnet


In [ ]:
def darknetClassifyAndShowResults(image_dir, image_file, dnet):
    imagec, blobc = darknetGetImageAndBlob(image_dir, image_file)

    # 7.4 - Definir a imagem lida como entrada para o processamento do modelo
    dnet = setNetInput(dnet, blobc)

    # 7.5 - Processar a inferência neste modelo, usando esta imagem
    outputs = processInference(dnet)
    #
    # output is a list with 1 element with shape (1, 1000, 1, 1)
    # reshape it to (1, 1000) - the shape expected in getOrderedPreds function

    out0 = outputs[0]
    dnet_preds = out0.reshape(out0.shape[1], out0.shape[0])
    dnet_ordered_inds = getOrderedPreds(dnet_preds, qtd_preds)

    printPredictions(dnet_ordered_inds, darknet_classes, dnet_preds, image_file)


    # 7.8 - Inserir na imagem o nome da classe que apresentou o maior índice de confiança
    image = putTextInImage(imagec, dnet_ordered_inds, darknet_classes, dnet_preds)

    # 7.9 - Exibir a imagem usando OpenCV
    showImage(image)


#### Inicialmente, utilizaremos o modelo darknet19 para efetuar classificação de imagens

In [ ]:
# Testes com darknet 19 para classificação
#
# definir o diretório onde está armazenado o modelo a ser usado
#
darknet_dir = models_path + 'Darknet/darknet_classif'


# definir o arquivo que contém os pesos pré-treinados deste modelo 
darknet_model = 'darknet19.weights'

# definir o arquivo que contém a 'configuração' deste modelo 
darknet_proto = 'darknet19.cfg'

# arquivo com os nomes das classes
darknet_classes_file = 'imagenet.shortnames.list'

darknet_classes = darknetGetClassNames(darknet_dir, darknet_classes_file) #getClassNames(darknet_dir, darknet_classes_file) #

#
dnet = loadDarknetClassifModel(darknet_dir, darknet_model, darknet_proto)

# determine the output layer
ln = dnet.getLayerNames()
ln = [ln[i[0] - 1] for i in dnet.getUnconnectedOutLayers()]

#



#### 7.12.1 - Agora, iremos classificar uma imagem usando o modelo já carregado em memória

In [ ]:
# definir a imagem a ser processada
image_file = 'eagle.png' 
  

darknetClassifyAndShowResults(image_dir, image_file, dnet)


### Atividade para o aluno
#### 7.12.2 - Exercício: utilizando o modelo já carregado, efetuar a classificação para as seguintes imagens
<ul>
<li>pexels-engin-akyurt-1769271.jpg</li>
<li>pexels-pascal-renet-1089304.jpg</li>
</ul>

### Atividade para o aluno
#### 7.12.3 - Exercício: tendo como base o exemplo acima, usar a rede darknet de referência para efetuar a classificação de imagens. O arquivo de classes será o mesmo já definido acima. Utilizar os seguintes arquivos referentes à configuração e aos pesos do modelo:
<ul>
<li>Modelo: 'darknet_reference.weights'</li>
<li>Config: 'darknet_reference.cfg'</li>
</ul>

### Atividade para o aluno
#### 7.12.4 - Exercício: utilizando o modelo darknet de referência já carregado, efetuar a classificação para as seguintes imagens
<ul>
<li>pexels-anel-rossouw-2558605.jpg</li>
<li>pexels-frank-grün-3757197.jpg</li>
</ul>

### Atividade para o aluno
#### 7.12.5 - Exercício: tendo como base o exemplo acima, usar a rede densenet 201 para efetuar a classificação de imagens. O arquivo de classes será o mesmo já definido acima. Utilizar os seguintes arquivos referentes à configuração e aos pesos do modelo:
<ul>
<li>Modelo: 'densenet201.weights'</li>
<li>Config: 'densenet201.cfg'</li>
</ul>

### Atividade para o aluno
#### 7.12.6 - Exercício: utilizando o modelo densenet 201 já carregado, efetuar a classificação para as seguintes imagens
<ul>
<li>pexels-anel-rossouw-2558605.jpg</li>
<li>pexels-pascal-renet-1089304.jpg</li>
<li>pexels-frank-grün-3757197.jpg</li>
</ul>

### Atividade para o aluno
#### 7.12.7 - Questões
<ul>
<li>Por que utilizamos o mesmo arquivo de classes nas atividades relacionadas aos modelos densenet 201, darknet19, darknet de referência?</li>
<li>Com base nas classificações obtidas para as diferentes imagens, é possível dizer se algum dos modelos vistos é melhor que os demais?</li>
</ul>

### Atividade para o aluno
#### 7.12.8 - Atividades extras
<ul>
<li>Efetuar a classificação de todas as imagens contidas no diretório e gravar em outra pasta de sua escolha as imagens obtidas ao final do processo de classificação.</li>
<li>Melhorar a atividade descrita acima, gravando os arquivos de imagens que possuem nomes com prefixo 'pexel', com nomes que representem suas respectivas classes. Por exemplo, o arquivo de nome cat.jpg será o arquivo que contém uma imagem cuja classificação com maior índice de confiança é 'cat'.</li>
</ul>